In [ ]:
import torch 
from torch import nn
import pandas as pd
import torch.optim as optim 
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms as T
import tensorflow as tf
import torch.nn.functional as F

In [ ]:
#GET Data
from google.colab import files
files.upload() #Upload kaggle.json here.

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d zalando-research/fashionmnist
!unzip -q fashionmnist.zip
!rm fashionmnist.zip

 89% 61.0M/68.8M [00:00<00:00, 198MB/s]
100% 68.8M/68.8M [00:00<00:00, 199MB/s]


In [ ]:
#Make sets
train_set = pd.read_csv('fashion-mnist_train.csv')
validation_set = train_set.iloc[50000:,]
test_set = pd.read_csv('fashion-mnist_test.csv')
train_set = train_set.iloc[:50000]

train_loader = DataLoader(dataset=train_set, batch_size=20, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=20, shuffle=True)

#train_dataset = torch.utils.data.TensorDataset()

trainset=tf.convert_to_tensor(train_set)
#print(trainset.type())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#train_set.type()

In [ ]:

#train_set = train_set[train_set.columns[1:]].values.reshape(50000, 28, 28)
#train_set.shape

In [ ]:
#Make model
#greyscale so in_channel is 1, out_channel chosen arbitrarily, same with kernel and stride

class CNN(nn.Module):
  #these are the default values, like a constructor
  def __init__(self, in_channels = 1 , out_channels = 100, kernel_size = 4, stride = 1): 
    super(CNN, self).__init__() 
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels=50, kernel_size =4, stride=1) #first layer
    self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size =2, stride =2)
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2)) #will cut output to half size
    #output has to be linear
    self.o = nn.Linear(10 * 6 * 6 , 10)

    # (B, 100, 25, 25) -> (B, 100 * 25 * 25)
    # input to linear: (B, D)
    # Linear(D, out_dim)
    # output from linear: (B, out_dim)

    # input to conv layer: (B, NC, h, w)
    # Conv2D()
    # output from conv layer: (B, out_channels, H_new, W_new)

  #got through all your layers
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.o(x)

    return x


model = CNN()

x = torch.rand(20, 1, 28, 28)
#print(x.type())
#print(x.shape[0], -1)
print(model(x).shape)
model(x)

torch.Size([20, 10])


tensor([[-0.0247,  0.0056,  0.0154, -0.0878,  0.2621,  0.1383,  0.0271, -0.0624,
         -0.1313,  0.0840],
        [-0.0655,  0.0271,  0.0106, -0.0845,  0.2934,  0.1674,  0.0032, -0.0781,
         -0.1100,  0.0746],
        [-0.0593,  0.0513,  0.0178, -0.0565,  0.2899,  0.1354, -0.0348, -0.0346,
         -0.1103,  0.1264],
        [-0.0174,  0.0409,  0.0293, -0.0939,  0.2420,  0.1486,  0.0403, -0.0581,
         -0.1129,  0.1256],
        [-0.0298,  0.0475,  0.0285, -0.0726,  0.2676,  0.1049,  0.0159, -0.0348,
         -0.1238,  0.0912],
        [-0.0690,  0.0319,  0.0571, -0.0762,  0.2505,  0.1380,  0.0351, -0.0003,
         -0.1353,  0.0750],
        [-0.0232,  0.0374,  0.0023, -0.1388,  0.2652,  0.1521,  0.0690, -0.0358,
         -0.1039,  0.1067],
        [-0.0593,  0.0537, -0.0024, -0.0705,  0.3045,  0.1221, -0.0207, -0.0741,
         -0.1178,  0.0815],
        [-0.0490,  0.0024, -0.0181, -0.0464,  0.2772,  0.0913,  0.0350, -0.0844,
         -0.1179,  0.0904],
        [-0.0355, -

In [ ]:
#from matplotlib import pyplot as plt

#plt.imshow(train_set[0])

In [ ]:
###REFORMAT TRAIN DATA ?
#class CustomImageDataset(Dataset):
#    def __init__(self, data):
#        self.data = data
#        self.transform = T.Compose([
#            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#        ])
#
#    def __len__(self):
#        return len(self.data)
#
#    def __getitem__(self, idx):
#        img = self.data[idx]  # (28, 28)
#        img = torch.Tensor(np.stack([img, img, img], axis=0))  # (3, 28, 28)
#        img = self.transform(img)  # (3, 28, 28)
#
#        return img
#
#trainset  = CustomImageDataset(train_set)

In [ ]:
#train_dataloader = DataLoader(trainset, 
#                              batch_size=20, 
#                              shuffle=True)

In [ ]:
##Training / instantiate some values
num_epochs = 2
learning_rate = 0.001

criterion = nn.CrossEntropyLoss() #Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
   for i, batch in enumerate(train_set):
        data, labels = batch 
        data = data.to(device)
        labels = labels.to(device)
     
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    #print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

ValueError: ignored